### Base de Dados

**Input**: 
* input/factors/YYYYMMDD.csv - fatores de todas ações listadas na NYSE na frequência de um minuto do dia DD-MM-YYYY.
* input/returns/YYYYMMDD.csv - retornos de todas ações listadas na NYSE na frequência de um minuto do dia DD-MM-YYYY.

**Output**: 
* output/data/tercile_breakpoint/equal_weighted/YYYYMMDD.csv - retornos de todas ações listadas na NYSE e portfólios baseado em fatores a partir desses ativos na frequência de um minuto do dia DD-MM-YYYY.

O objetivo desse notebook é juntar duas bases de dados, uma contendo dados de retornos das ações e outro contendo os decis de cada ação. A partir desse merge, vamos criar portfólios Equal Weighted dos fatores utilizando os tercis como Breakpoints.

In [1]:
# pacotes
import numpy as np
import pandas as pd

In [2]:
# oculta mensagens de avisos
import warnings
warnings.filterwarnings("ignore")

In [3]:
# pd.set_option('display.max_columns', None)

In [4]:
# pd.set_option('display.max_rows', None)

### Functions

Como a metodologia para se criar o portfólio é a mesma, isso é, comprado em todos ativos acima da mediana e vendido em todos ativos abaixo da mediana, vamos criar funções que façam isso recebendo a coluna do fator em questão como parâmetro.

In [5]:
def portfolio_position(col, df_factors, df_returns):
    long_position = []      # posição comprado
    short_position = []     # posição vendido  

    # primeiro, precisamos dropar as linhas tais que tem valor NaN no parâmetro de localidade do fator
    temp = df_factors[df_factors[col].notna()]
    """
    Criamos um loop com o critério:
    tercil de cima: '7' estritamente ('8' fracamente), então o portfolio assume a posição comprado
    tercil de baixo: '4' estritamente ('3' fracamente), então o portfolio assume a posição vendido
    """
    for permno in temp.index:
        if temp.loc[permno][col] >= 8:
            long_position.append(temp.loc[permno]['TAQ_TICKER'])
        elif temp.loc[permno][col] <= 3:
            short_position.append(temp.loc[permno]['TAQ_TICKER'])
    """
    agora precisamos criar dois loops, um para cada lista:
    o loop deve verificar se o ticker dentro da lista é uma das colunas da base de retornos,
    isso é, verifica se temos dados de retornos para essa ação.
    """

    drop_long_position = []
    drop_short_position = []

    for ticker in long_position:
        if ticker not in df_returns.columns:
            drop_long_position.append(ticker)

    for ticker in drop_long_position:
        long_position.remove(ticker)

    for ticker in short_position:   
        if ticker not in df_returns.columns:
            drop_short_position.append(ticker)

    for ticker in drop_short_position:
        short_position.remove(ticker)

    return(long_position, short_position)

In [6]:
def portfolio_formation(col, df, df_factors, df_returns):
    # usaremos a função portfolio_position() para obtermos as firmas na posição comprada e vendida
    long_position, short_position = portfolio_position(col, df_factors, df_returns)
    # a nova coluna do fator se chamará:
    new_col = col[6].upper() + col[7:] + ' Factor'

    """
    O portfolio é formado subtraindo os retornos médio (equal weighted) das ações em short_position
    dos retornos médio (equal weighted) das ações em long_position
    """
    df[new_col] = df_returns[long_position].mean(axis=1) - df_returns[short_position].mean(axis=1)
    

### Data Generator Loop

Esse loop deverá gerar dados diários (um .csv para cada dia) com todos retornos e todos portfolios (para cada fator).

In [7]:
# precisamos criar um range de datas para o período de dados que temos
bdates = pd.bdate_range('2005-01-01', '2019-12-31')
bdates_ = []

# precisamos torná-los da maneira que os csv's estão nomeados
for date in bdates:
    day = str(date)[:4] + str(date)[5:7] + str(date)[8:10]
    bdates_.append(day)

In [8]:
for day in bdates_:
    try:
        # factors dataframe
        factors_path = f'../../../input/factors/{day}.csv'
        factors = pd.read_csv(factors_path, index_col=0)
        # precisamos dropar todas ações que não tem TAQ_TICKER, pois é a única variável q conseguimos ligar na outra base
        factors = factors[factors['TAQ_TICKER'] != '<undefined>']
        
        # returns dataframe
        returns_path = f'../../../input/returns/{day}.csv'
        returns = pd.read_csv(returns_path, index_col=0)
        
        # criando um dataframe para guardar os portfolios dos fatores
        portfolio = pd.DataFrame(index=returns.index)

        # agora precisamos fazer o loop que para cada coluna de pNYSE_factors, forma-se um portfolio
        pNYSE_factors = factors.columns[4:]
        for pNYSE in pNYSE_factors:
            portfolio_formation(pNYSE, portfolio, factors, returns)

        # precisamos agora fazer a conversão do dataframe para csv
        output_path = f'../../../output/data/tercile_breakpoint/equal_weighted/{day}.csv'
        portfolio.to_csv(output_path, sep=',', encoding='utf-8')
    except:
        pass